## Скрипт для создания заявок Subzero на несколько аккаунтов МС


In [20]:
# ! pip install psycopg2-binary
# ! pip install snowflake-connector-python
# !pip install ipython-sql
# pip install tqdm

from tqdm import tqdm
import pandas as pd
import requests
import json
import sqlalchemy
from concurrent.futures import ThreadPoolExecutor

con = sqlalchemy.create_engine('postgresql://postgres:@127.0.0.1:8181/lognex')

In [21]:
# Задание переменных:
namespace = 5  # Неймспейс
ticket_number = 1  # Количество заявок на 1 аккаунт
thread_number = 20  # Количество потоков

url = f'https://subzero-billing-{namespace}.testms-test.lognex.ru/api/clinton/1.0/ticket'
headers = {'Content-Type': 'application/json'}

In [22]:
# Фукнция для запроса к базе
def sql_go(request):
    return pd.read_sql(sqlalchemy.text(request), con)

In [4]:
# Вручную выгруженный файл из базы с аккаунтами
# accounts = pd.read_csv('lognex_billing_billingaccount.csv',header=None)
# accounts=accounts.values.tolist()

In [23]:
# Запрос
sql = '''
            select id from billing.billingaccount
        '''

In [24]:
accounts = sql_go(sql)
# print(accounts)
# print(accounts.info())

## Функция для скрипта

In [25]:
def bomber_to_many(row):
    counter = 0
    with open('json3.json', 'r') as json3:
        sub_data = json.load(json3)
    sub_data['accountId'] = str(row["id"])

    with requests.Session() as s:
        for ticket in tqdm(range(ticket_number), desc=f'Making requests for {row}'):
            counter += 1
            try:
                with s.post(url, data=json.dumps(sub_data), headers=headers) as r:
                    if r.ok:
                        print(f'Заявка для {row} создана', '\n')
                    else:
                        print(f'Заявка для {row} НЕ создана'+str(r))
            except Exception as e:
                print('Error occurred while making request:', e)
    return counter

# Создание пула потоков
with ThreadPoolExecutor(max_workers=thread_number) as executor:
    # Запуск потоков
    results = [executor.submit(bomber_to_many, row) for index, row in accounts.iterrows()]
    total_tickets = sum(result.result() for result in tqdm(results, desc='Aggregating results'))

print('------------------------------------------------')
print(f'итого, создано заявок: {total_tickets}')

Making requests for id    4615c8f6-0e7b-11e2-06e0-3c4a92f3a0a7
Name: 0, dtype: object:   0%|          | 0/1 [00:00<?, ?it/s]

Making requests for id    fbc15f55-8690-11ed-0a81-062400000000
Name: 4, dtype: object:   0%|          | 0/1 [00:00<?, ?it/s]
Making requests for id    7e9629bb-85c2-11ed-0a81-08d900000028
Name: 2, dtype: object:   0%|          | 0/1 [00:00<?, ?it/s]


Making requests for id    54a3aa65-86bf-11ed-0a81-062400000056
Name: 5, dtype: object:   0%|          | 0/1 [00:00<?, ?it/s]



Making requests for id    9f0f92b0-85c4-11ed-0a81-08d900000040
Name: 3, dtype: object:   0%|          | 0/1 [00:00<?, ?it/s]




Aggregating results:   0%|          | 0/9 [00:00<?, ?it/s]






Making requests for id    f007f07c-85b9-11ed-0a81-08d900000002
Name: 6, dtype: object:   0%|          | 0/1 [00:00<?, ?it/s]





Making requests for id    5e99ece8-86bf-11ed-0a81-06240000005f
Name: 7, dtype: object:   0%|          | 0/1 [00:00<?, ?it/s]








Making requests for id    21f9e601-85

Заявка для id    4615c8f6-0e7b-11e2-06e0-3c4a92f3a0a7
Name: 0, dtype: object НЕ создана<Response [500]>
Заявка для id    54a3aa65-86bf-11ed-0a81-062400000056
Name: 5, dtype: object создана 






Making requests for id    fbc15f55-8690-11ed-0a81-062400000000
Making requests for id    fbc15f55-8690-11ed-0a81-06240000000033it/s]
Name: 4, dtype: object: 100%|██████████| 1/1 [00:00<00:00,  1.32it/s]




Making requests for id    9f0f92b0-85c4-11ed-0a81-08d900000040
Name: 3, dtype: object: 100%|██████████| 1/1 [00:00<00:00,  1.24it/s]
Making requests for id    7e9629bb-85c2-11ed-0a81-08d900000028
Name: 2, dtype: object: 100%|██████████| 1/1 [00:00<00:00,  1.22it/s]








Making requests for id    21f9e601-85c5-11ed-0a81-08d90000004f
Making requests for id    9f0f92b0-85c4-11ed-0a81-08d90000004027it/s]
Name: 3, dtype: object: 100%|██████████| 1/1 [00:00<00:00,  1.22it/s]
Making requests for id    7e9629bb-85c2-11ed-0a81-08d900000028
Name: 2, dtype: object: 100%|██████████| 1/1 [00:00<00:00,  1.20it/s]
Making requests for id    21f9e601-85c5-11ed-0a81-08d90000004f
Name: 1, dtype: object: 100%|██████████| 1/1 [00:00<00:00,  1.25it/s]





Aggregating results:  22%|██▏       | 2/9 

Заявка для id    fbc15f55-8690-11ed-0a81-062400000000
Name: 4, dtype: object создана 

Заявка для id    9f0f92b0-85c4-11ed-0a81-08d900000040
Name: 3, dtype: object создана 

Заявка для id    7e9629bb-85c2-11ed-0a81-08d900000028
Name: 2, dtype: object создана 

Заявка для id    21f9e601-85c5-11ed-0a81-08d90000004f
Name: 1, dtype: object создана 

Заявка для id    f007f07c-85b9-11ed-0a81-08d900000002
Name: 6, dtype: object создана 

Заявка для id    5e99ece8-86bf-11ed-0a81-06240000005f
Name: 7, dtype: object создана 












Making requests for id    8cf46406-85c1-11ed-0a81-08d90000001e
Making requests for id    8cf46406-85c1-11ed-0a81-08d90000001e02it/s]
Name: 8, dtype: object: 100%|██████████| 1/1 [00:00<00:00,  1.02it/s]





Aggregating results: 100%|██████████| 9/9 [00:01<00:00,  8.92it/s]


Заявка для id    8cf46406-85c1-11ed-0a81-08d90000001e
Name: 8, dtype: object создана 

------------------------------------------------
итого, создано заявок: 9
